# Setup

In [1]:
%pip install spuco

In [2]:
import torch

device = torch.device("cpu")

In [3]:
from spuco.utils import set_seed

set_seed(0)

# Dataset Creation

In [4]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train",
    label_noise=0.001
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()


In [5]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]

valset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()


In [6]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [7]:
from spuco.models import model_factory
from spuco.robust_train import ERM
from torch.optim import SGD
import numpy as np
from spuco.robust_train import CustomSampleERM
from spuco.utils import Trainer
from spuco.group_inference import JTTInference
from spuco.robust_train import CustomSampleERM
from spuco.evaluate import Evaluator
import random
random.seed(10)

# Train using JTT

In [8]:
#shuffle training set
print('Total trainset size: ' + str(len(trainset)))
shuffled_indices = np.arange(0, len(trainset))
random.shuffle(shuffled_indices)

Total trainset size: 48004


## Group Inference

In [9]:
def gi_subset(num_partitions, partition_id, trainset, device, shuffled_indices, verbose=False):
  #generate subset index
  partition_len = int(len(trainset)/num_partitions)
  train_index = shuffled_indices[partition_len*partition_id:partition_len*partition_id+partition_len]
  train_index = train_index.tolist()
  #print(train_index)
  #train on subset
  infer_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
  gi_trainer = CustomSampleERM(
      model=infer_model,
      trainset=trainset,
      batch_size=64,
      optimizer=SGD(infer_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
      num_epochs=2,
      indices=train_index,
      device=device,
      verbose=verbose
  )
  gi_trainer.train()
  #get labels
  gi_trainer_results = Trainer(
    model=infer_model,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(infer_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=verbose
  )
  output = gi_trainer_results.get_trainset_outputs()
  max_output = torch.max(output, dim=1)
  pred_labels = max_output.indices
  pred_labels = pred_labels.tolist()
  actual_labels = trainset.labels
  #jtt inference on subset
  jtt = JTTInference([pred_labels[i] for i in train_index], [actual_labels[i] for i in train_index])
  gi = jtt.infer_groups()
  nominal_upsample = len(gi[(0, 0)])/len(gi[(0, 1)])
  if verbose:
    print('\nNominal Upsample: ' + str(nominal_upsample))
    print('Majority class size: ' + str(len(gi[(0, 0)])))
    print('Minority class size: ' + str(len(gi[(0, 1)])))
  #return majority, minority group indices
  majority = [train_index[i] for i in gi[(0, 0)]]
  minority = [train_index[i] for i in gi[(0, 1)]]
  return majority, minority

In [10]:
def full_gi(num_partitions, trainset, device, shuffled_indices, verbose=False):
  maj, min = gi_subset(num_partitions, 0, trainset, device, shuffled_indices, verbose)
  for i in range(1, num_partitions):
    maj_temp, min_temp = gi_subset(num_partitions, i, trainset, device, shuffled_indices, verbose)
    maj.extend(maj_temp)
    min.extend(min_temp)
  print('Majority class size: ' + str(len(maj)))
  print('Minority class size: ' + str(len(min)))
  upsampled_indices = [i for i in min for x in range(100)]
  upsampled_indices.extend(maj)
  print('Upsampled length: ' + str(len(upsampled_indices)))
  return upsampled_indices

In [11]:
#upsampled_indices = full_gi(2, trainset, device, shuffled_indices, verbose=True)

## Robust Training

In [12]:
def robust_train(trainset, testset, device, upsampled_indices, verbose=False):
  #robust training
  robust_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
  rt_erm = CustomSampleERM(
      model=robust_model,
      trainset=trainset,
      batch_size=64,
      optimizer=SGD(robust_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
      num_epochs=3,
      indices=upsampled_indices,
      device=device,
      verbose=verbose
  )
  rt_erm.train()
  #evaluate robust model
  rt_evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=robust_model,
    device=device,
    verbose=verbose
  )
  rt_evaluator.evaluate()
  wg_acc = rt_evaluator.worst_group_accuracy
  avg_acc = rt_evaluator.average_accuracy
  spur_acc = rt_evaluator.evaluate_spurious_attribute_prediction()
  print('\nWorst group accuracy: ' + str(wg_acc))
  print('Average accuracy: ' + str(avg_acc))
  print('Spurious attribute accuracy: ' + str(spur_acc))
  return wg_acc, avg_acc, spur_acc

In [13]:
#robust_train(trainset, testset, device, upsampled_indices)

## Sweep Num Partitions

In [14]:
shuffled_indices = np.arange(0, len(trainset))
random.shuffle(shuffled_indices)
upsampled_indices = full_gi(8, trainset, device, shuffled_indices)
wg, avg, spur = robust_train(trainset, testset, device, upsampled_indices)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Majority class size: 47709
Minority class size: 291
Upsampled length: 76809


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:07<00:00,  3.18it/s]



Worst group accuracy: ((4, 2), 26.952141057934508)
Average accuracy: 99.09204311981746
Spurious attribute accuracy: 19.96
